## Thermodynamic Property Prediction for H, S, Tc
1. Generates models/predictions for H, S, Tc
2. Uses transfer learning to predict Tc
3. Calculates a prediction for Tc using H and S predictions and the Tc formula

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import joblib
import os

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, WhiteKernel, Matern, RationalQuadratic)
from sklearn.kernel_ridge import KernelRidge
from sklearn.utils import resample

from xgboost import XGBRegressor
from collections import Counter

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, rdMolDescriptors
from rdkit.ML.Cluster import Butina

from great_tables import GT, loc, style
import polars as pl

import shap

/Users/skoz/miniconda3/envs/polyenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
plt.style.use("presentation.mplstyle")
mpl.rcParams['axes.prop_cycle'] = cycler(color=['#5c3c8b', '#92c36d', '#ee9432', '#496391', '#85a5cd', '#FDF3CC'])
plotsize = 5

### Define Nested CV Parameters

In [5]:
file_info = {
    "enthalpy": "2_split_datasets/enthalpy_80_MI_reduced.csv",
    "entropy": "2_split_datasets/entropy_80_MI_reduced.csv",
}

feature_sets = [
    (name, pd.read_csv(path, index_col=0))
    for name, path in file_info.items()
]

In [6]:
# store column indices for use in the loop
target_idx = 0
phase_split_idx = 1
category_idx = 2
smiles_idx = 3
global_test_idx = 4
first_feature_idx = 5

In [5]:
models = [
    ('gpr', GaussianProcessRegressor(), {
        "model__kernel": [], 
    }),
    ('rf', RandomForestRegressor(), {
        'model__n_estimators': [25, 50, 100],
        'model__min_samples_split': [2, 5]
    }),
    ('xgb', XGBRegressor(), {
        'model__n_estimators': [50, 100],
        'model__max_depth': [3, 6],
        'model__learning_rate': [0.01, 0.1],
    }),
    ('svr', SVR(), {
        'model__C': [0.1, 1, 10],
        'model__gamma': ['scale', 'auto'],
        'model__kernel': ['rbf']
    }),
    ('krr', KernelRidge(), {
        'model__alpha': [1e-3, 1e-2, 1e-1, 1.0],
        'model__kernel': ['rbf', 'linear', 'poly'],
        'model__gamma': [0.01, 0.1, 1],
    }),
]

In [7]:
outer_splits = 10
inner_splits = 5
results = []

model_dir = "5_saved_models"
output_dir = "5_images_and_csvs"

os.makedirs(model_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

### Nested CV and Evaluation Loop: initial models for H, S, Tc

In [7]:
def signed_log_transform(y):
    return np.sign(y) * np.log1p(np.abs(y))

def signed_log_inverse(z):
    return np.sign(z) * (np.expm1(np.abs(z)))

In [8]:
for split_type in ['phase', 'chemistry_cluster']:
    print(f"\n***** Split by {split_type} *****")

    for target_name, dataframe in feature_sets:
        print(f"\nTarget Set: {target_name}")

        # Extract features, target, and grouping columns
        X = dataframe.iloc[:, first_feature_idx:]
        y_raw = dataframe.iloc[:, target_idx]

        # Drop any columns containing "PEP" in X
        X = X.loc[:, ~X.columns.str.contains("PEP")]
        
        # If log in name, apply signed-log transform
        if "log" in target_name.lower():
            y = signed_log_transform(y_raw)
            use_log = True
        else:
            y = y_raw
            use_log = False
        
        phase = dataframe.iloc[:, phase_split_idx]
        smiles = dataframe.iloc[:, smiles_idx]
        global_test = dataframe.iloc[:, global_test_idx]

        # Pull out X_global_test and y_global_test
        X_global_test = X.loc[global_test]
        y_global_test = y_raw.loc[global_test]

        # Pull out training data to use in the nested CV scheme
        X_train = X.loc[~global_test]
        y_train = y.loc[~global_test]
        phase_train = phase.loc[~global_test]
        smiles_train = smiles.loc[~global_test]
                
        # Based on split type, create outer CV splits
        if split_type == 'phase':
            outer_cv = list(StratifiedKFold(n_splits=outer_splits, shuffle=True, random_state=42).split(X_train, phase_train))
        elif split_type == 'chemistry_cluster':
            # re-cluster now that there's different data
            # Generate fingerprints
            mols = [Chem.MolFromSmiles(s) for s in smiles_train]
            fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048) for m in mols]

            n = len(fps)
            dists = []
            for i in range(1, n):
                sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[:i])
                dists.extend([1 - x for x in sims]) 

            # Cluster using Butina
            clusters = Butina.ClusterData(dists, n, 0.2, isDistData=True) 
            cluster_map = {i: [X_train.index[j] for j in cluster] for i, cluster in enumerate(clusters)}

            cluster_labels = np.full(len(X_train), -1, dtype=int)
            for cid, cluster in enumerate(clusters):
                for j in cluster:
                    cluster_labels[j] = cid

            outer_cv = list(GroupKFold(n_splits=outer_splits).split(X_train, groups=cluster_labels))        

        model_scores = []

        for name, model, param_grid in models:
            print(f"\nModel: {name}")
            if name in ['svr', 'krr', 'gpr']:
                pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])
            else:
                pipe = Pipeline([('model', model)])

            # Set GPR params based on feature set
            if name == "gpr":
                global_y_mean = np.mean(y_train)
                global_noise_est = np.std(y_train) / 3
                rbf = global_y_mean**2 * RBF(length_scale=10.0, length_scale_bounds=(1e-2, 1e3)) + \
                    WhiteKernel(noise_level=global_noise_est**2, noise_level_bounds=(1e-6, 10))
                rq = global_y_mean**2 * RationalQuadratic(length_scale=10.0) + \
                    WhiteKernel(noise_level=global_noise_est**2, noise_level_bounds=(1e-6, 10))
                matern = global_y_mean**2 * Matern(length_scale=10.0, nu=1.5) + \
                        WhiteKernel(noise_level=global_noise_est**2, noise_level_bounds=(1e-6, 10))

                param_grid['model__kernel'] = [rbf, rq, matern]
            
            cv_r2_scores = []
            cv_mae_scores = []
            best_params_per_fold = []

            # OUTER CV LOOP
            for fold_idx, (outer_train_idx, outer_val_idx) in enumerate(outer_cv):
                X_outer_train = X_train.iloc[outer_train_idx]
                X_outer_val = X_train.iloc[outer_val_idx] 
                y_outer_train = y_train.iloc[outer_train_idx]
                y_outer_val = y_train.iloc[outer_val_idx]

                # INNER CV LOOP
                if param_grid:
                    # based on spilt type, come up with inner CV folds
                    if split_type == 'phase':
                        phase_outer_train = phase_train.iloc[outer_train_idx]

                        inner_cv_splits = list(StratifiedKFold(
                            n_splits=inner_splits, 
                            shuffle=True, 
                            random_state=43
                        ).split(X_outer_train, phase_outer_train))

                        grid = GridSearchCV(
                            pipe, param_grid, 
                            cv=inner_cv_splits, 
                            scoring='neg_mean_absolute_error'
                        )

                    elif split_type == 'chemistry_cluster':
                        cluster_outer_train = cluster_labels[outer_train_idx]
                        inner_cv_splits = list(GroupKFold(
                            n_splits=inner_splits
                        ).split(X_outer_train, groups=cluster_outer_train))
                        
                        grid = GridSearchCV(
                            pipe, param_grid,
                            cv=inner_cv_splits, 
                            scoring='neg_mean_absolute_error'
                        )

                    grid.fit(X_outer_train, y_outer_train)
                    best_estimator = grid.best_estimator_
                    best_params_per_fold.append(grid.best_params_)
                    
                else:
                    # No hyperparameters to tune
                    best_estimator = pipe.fit(X_outer_train, y_outer_train)
                    best_params_per_fold.append({})

                y_outer_pred = best_estimator.predict(X_outer_val)
                if use_log:
                    y_outer_pred = signed_log_inverse(y_outer_pred)
                    y_outer_val_for_score = signed_log_inverse(y_outer_val)
                else:
                    y_outer_val_for_score = y_outer_val

                r2 = r2_score(y_outer_val_for_score, y_outer_pred)
                mae_score = mean_absolute_error(y_outer_val_for_score, y_outer_pred)

                cv_r2_scores.append(r2)
                cv_mae_scores.append(mae_score)

            mean_r2 = np.mean(cv_r2_scores)
            std_r2 = np.std(cv_r2_scores)
            
            mean_mae = np.mean(cv_mae_scores)
            std_mae = np.std(cv_mae_scores)

            print(f"Nested CV MAE: {mean_mae:.2f} ± {std_mae:.2f}")
            print(f"Nested CV R²: {mean_r2:.2f} ± {std_r2:.2f}")

            if param_grid and best_params_per_fold:
                # Find most frequent parameter combination
                param_strings = [str(sorted(params.items())) for params in best_params_per_fold]
                most_common_params = Counter(param_strings).most_common(1)[0][0]
                # Convert back to dict (this is a bit hacky but works)
                most_common_dict = dict(eval(most_common_params))
                
                print(f"Most common parameters: {most_common_dict}")
                
                # Set the pipeline with these parameters
                final_model = pipe.set_params(**most_common_dict)
                final_model.fit(X_train, y_train)

            # Make predictions on global models (account for log prediction)
            y_pred = final_model.predict(X_global_test)
            if use_log:
                y_pred = signed_log_inverse(y_pred)

            test_mae = mean_absolute_error(y_global_test, y_pred)
            test_r2 = r2_score(y_global_test, y_pred)
            print(f"Test MAE: {test_mae:.2f}, Test R²: {test_r2:.2f}")

            model_scores.append((name, test_mae, test_r2))
            results.append({
                "target_Name": target_name,
                "model": name,
                "nested_cv_mae": mean_mae,
                "nested_cv_std": std_mae,
                "nested_cv_r2_mean": mean_r2,
                "nested_cv_r2_std": std_r2,
                "test_mae": test_mae,
                "test_r2": test_r2,
                "split_type": split_type,
            })

            joblib.dump(final_model, f"{model_dir}/{target_name}_{name}_{split_type}.joblib")


***** Split by phase *****

Target Set: enthalpy

Model: gpr
Nested CV MAE: 2.58 ± 0.40
Nested CV R²: 0.71 ± 0.09
Most common parameters: {'model__kernel': 8.69**2 * RationalQuadratic(alpha=1, length_scale=10) + WhiteKernel(noise_level=5.97)}
Test MAE: 2.13, Test R²: 0.83

Model: rf
Nested CV MAE: 2.43 ± 0.37
Nested CV R²: 0.71 ± 0.10
Most common parameters: {'model__min_samples_split': 2, 'model__n_estimators': 100}
Test MAE: 1.27, Test R²: 0.92

Model: xgb
Nested CV MAE: 2.43 ± 0.40
Nested CV R²: 0.71 ± 0.10
Most common parameters: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 50}
Test MAE: 1.54, Test R²: 0.90

Model: svr
Nested CV MAE: 2.79 ± 0.44
Nested CV R²: 0.60 ± 0.12
Most common parameters: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
Test MAE: 2.01, Test R²: 0.77

Model: krr
Nested CV MAE: 2.69 ± 0.47
Nested CV R²: 0.69 ± 0.11
Most common parameters: {'model__alpha': 0.01, 'model__gamma': 0.01, 'model__kernel': 'rbf'}
Test M

[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerator
[11:25:57] DEPRECATION WARNING: please use MorganGenerat

Nested CV MAE: 3.21 ± 0.79
Nested CV R²: 0.58 ± 0.18
Most common parameters: {'model__kernel': 8.69**2 * RationalQuadratic(alpha=1, length_scale=10) + WhiteKernel(noise_level=5.97)}
Test MAE: 2.13, Test R²: 0.83

Model: rf
Nested CV MAE: 2.75 ± 0.47
Nested CV R²: 0.67 ± 0.11
Most common parameters: {'model__min_samples_split': 2, 'model__n_estimators': 100}
Test MAE: 1.31, Test R²: 0.92

Model: xgb
Nested CV MAE: 2.81 ± 0.45
Nested CV R²: 0.66 ± 0.10
Most common parameters: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 50}
Test MAE: 1.54, Test R²: 0.90

Model: svr
Nested CV MAE: 3.17 ± 0.52
Nested CV R²: 0.53 ± 0.10
Most common parameters: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
Test MAE: 2.01, Test R²: 0.77

Model: krr
Nested CV MAE: 3.67 ± 1.93
Nested CV R²: 0.24 ± 1.06
Most common parameters: {'model__alpha': 0.1, 'model__gamma': 0.01, 'model__kernel': 'rbf'}
Test MAE: 1.95, Test R²: 0.85

Target Set: entropy

Model: gpr


[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerator
[11:30:57] DEPRECATION WARNING: please use MorganGenerat

Nested CV MAE: 4.21 ± 0.61
Nested CV R²: 0.76 ± 0.07
Most common parameters: {'model__kernel': 14.1**2 * Matern(length_scale=10, nu=1.5) + WhiteKernel(noise_level=15.6)}
Test MAE: 3.36, Test R²: 0.77

Model: rf
Nested CV MAE: 4.14 ± 0.79
Nested CV R²: 0.76 ± 0.08
Most common parameters: {'model__min_samples_split': 2, 'model__n_estimators': 50}
Test MAE: 3.54, Test R²: 0.76

Model: xgb
Nested CV MAE: 4.26 ± 0.90
Nested CV R²: 0.75 ± 0.09
Most common parameters: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 100}
Test MAE: 4.06, Test R²: 0.69

Model: svr
Nested CV MAE: 4.70 ± 0.85
Nested CV R²: 0.70 ± 0.10
Most common parameters: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
Test MAE: 3.36, Test R²: 0.80

Model: krr
Nested CV MAE: 7.54 ± 9.01
Nested CV R²: -32.46 ± 99.55
Most common parameters: {'model__alpha': 0.1, 'model__gamma': 0.01, 'model__kernel': 'rbf'}
Test MAE: 3.91, Test R²: 0.67


In [9]:
df = pd.DataFrame(results)
df = df.round(2)
df['test_mae'] = np.where(df['target_Name'] == 'tc', df['test_mae'].round(), df['test_mae'])

df.to_csv(f"{output_dir}/model_results_summary.csv", index=False)

In [10]:
df = pd.read_csv(f"{output_dir}/model_results_summary.csv")

### Flag best models in directory

In [11]:
for split_type in ['phase', 'chemistry_cluster']:
    for target_name, _ in feature_sets:
        # find the best model with the lowest MAE
        subset = df[(df['split_type'] == split_type) & (df['target_Name'] == target_name)]
        best_model_row = subset.sort_values(by='test_mae').head(1)
        
        if not best_model_row.empty:
            best_model_name = best_model_row['model'].values[0]
            model_path = f"{model_dir}/{target_name}_{best_model_name}_{split_type}.joblib"
            if os.path.exists(model_path):
                new_model_path = f"{model_dir}/{target_name}_{best_model_name}_{split_type}_BEST.joblib"
                os.rename(model_path, new_model_path)

### Generate table

In [12]:
data = {
    "Model": [item.upper() for item in df['model'][:len(models)].tolist()],
    
    "Split by Chemistry.ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    # "Split by Chemistry.Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    # "Split by Chemistry.Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    # "Split by Chemistry.log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    # "Split by Chemistry.log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    

    "Split by Phase.ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    # "Split by Phase.Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    # "Split by Phase.Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    # "Split by Phase.log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    # "Split by Phase.log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

for split in ['Chemistry', 'Phase']:
    for prop in ['ΔHₚ', 'ΔSₚ', 'Tc', 'log Tc']:
            for metric in ['MAE', 'R²']:
                gt = gt.fmt_number(columns=f"Split by {split}.{prop}.{metric}", decimals=2, drop_trailing_zeros=False)

gt = gt.fmt_integer(columns="Split by Chemistry.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Chemistry.log Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.log Tc.MAE", use_seps=False)

gt = gt.opt_horizontal_padding(scale=2.0)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#C0ABDC",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
)

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/model_result_table.png")

GT(_tbl_data=  Model  Split by Chemistry.ΔHₚ.MAE  Split by Chemistry.ΔHₚ.R²  \
0   GPR                        2.13                       0.83   
1    RF                        1.31                       0.92   
2   XGB                        1.54                       0.90   
3   SVR                        2.01                       0.77   
4   KRR                        1.95                       0.85   

   Split by Chemistry.ΔSₚ.MAE  Split by Chemistry.ΔSₚ.R²  \
0                        3.36                       0.77   
1                        3.54                       0.76   
2                        4.06                       0.69   
3                        3.36                       0.80   
4                        3.91                       0.67   

   Split by Phase.ΔHₚ.MAE  Split by Phase.ΔHₚ.R²  Split by Phase.ΔSₚ.MAE  \
0                    2.13                   0.83                    3.36   
1                    1.27                   0.92                    3.55   
2                    1.54                   0.90                    3.64   
3                    2.01                   0.77                    3.36   
4                    2.05                   0.82                    3.91   

   Split by Phase.ΔSₚ.R²  
0                   0.77  
1                   0.77  
2                   0.72  
3                   0.80  
4                   0.67  , _body=<great_tables._gt_data.Body object at 0x16a58e2d0>, _boxhead=Boxhead([ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔSₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Phase.ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase.ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Phase.ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase.ΔSₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x167447dd0>, _spanners=Spanners([SpannerInfo(spanner_id='ΔHₚ', spanner_level=0, spanner_label='ΔHₚ', spanner_units=None, spanner_pattern=None, vars=['Split by Chemistry.ΔHₚ.MAE', 'Split by Chemistry.ΔHₚ.R²', 'Split by Phase.ΔHₚ.MAE', 'Split by Phase.ΔHₚ.R²'], built=None), SpannerInfo(spanner_id='ΔSₚ', spanner_level=0, spanner_label='ΔSₚ', spanner_units=None, spanner_pattern=None, vars=['Split by Chemistry.ΔSₚ.MAE', 'Split by Chemistry.ΔSₚ.R²', 'Split by Phase.ΔSₚ.MAE', 'Split by Phase.ΔSₚ.R²'], built=None), SpannerInfo(spanner_id='Split by Chemistry', spanner_level=1, spanner_label='Split by Chemistry', spanner_units=None, spanner_pattern=None, vars=['Split by Chemistry.ΔHₚ.MAE', 'Split by Chemistry.ΔHₚ.R²', 'Split by Chemistry.ΔSₚ.MAE', 'Split by Chemistry.ΔSₚ.R²'], built=None), SpannerInfo(spanner_id='Split by Phase', spanner_level=1, spanner_label='Split by Phase', spanner_units=None, spanner_pattern=None, vars=['Split by Phase.ΔHₚ.MAE', 'Split by Phase.ΔHₚ.R²', 'Split by Phase.ΔSₚ.MAE', 'Split by Phase.ΔSₚ.R²'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1674457f0>, _formats=[<great_tables._gt_data.Forma

In [11]:
data = {
    "Property (units)": ["ΔHₚ (kcal/mol)", "ΔSₚ (cal/mol/K)"],
    
    "All Features.MAE": [1.27,3.24],
    "All Features.R²": [0.91,0.80],
    "Without PEP.MAE": [1.31,3.36],
    "Without PEP.R²": [0.90,0.80],
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

for prop in ['ΔHₚ', 'ΔSₚ', 'Tc', 'log Tc', 'Tc TL']:
        for metric in ['MAE', 'R²']:
            gt = gt.fmt_number(columns=f"{prop}.{metric}", decimals=2, drop_trailing_zeros=False)

gt = gt.fmt_integer(columns="Tc TL.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="log Tc.MAE", use_seps=False)

gt = gt.opt_horizontal_padding(scale=2.0)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#C0ABDC",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
)

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/comparison_result_table_chem_only.png")

GT(_tbl_data=  Property (units)  All Features.MAE  All Features.R²  Without PEP.MAE  \
0   ΔHₚ (kcal/mol)              1.27             0.91             1.31   
1  ΔSₚ (cal/mol/K)              3.24             0.80             3.36   

   Without PEP.R²  
0             0.9  
1             0.8  , _body=<great_tables._gt_data.Body object at 0x17975ca10>, _boxhead=Boxhead([ColInfo(var='Property (units)', type=<ColInfoTypeEnum.default: 1>, column_label='Property (units)', column_align='center', column_width=None), ColInfo(var='All Features.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='All Features.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Without PEP.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Without PEP.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x17c09cfe0>, _spanners=Spanners([SpannerInfo(spanner_id='All Features', spanner_level=0, spanner_label='All Features', spanner_units=None, spanner_pattern=None, vars=['All Features.MAE', 'All Features.R²'], built=None), SpannerInfo(spanner_id='Without PEP', spanner_level=0, spanner_label='Without PEP', spanner_units=None, spanner_pattern=None, vars=['Without PEP.MAE', 'Without PEP.R²'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x17c09c800>, _formats=[<great_tables._gt_data.FormatInfo object at 0x17c09cce0>, <great_tables._gt_data.FormatInfo object at 0x17c09cc20>, <great_tables._gt_data.FormatInfo object at 0x17c09c8c0>, <great_tables._gt_data.FormatInfo object at 0x17c09d190>, <great_tables._gt_data.FormatInfo object at 0x17c09d340>, <great_tables._gt_data.FormatInfo object at 0x17c09d3a0>, <great_tables._gt_data.FormatInfo object at 0x17c09d670>, <great_tables._gt_data.FormatInfo object at 0x17c09d760>, <great_tables._gt_data.FormatInfo object at 0x17c09d850>, <great_tables._gt_data.FormatInfo object at 0x17c09d940>, <great_tables._gt_data.FormatInfo object at 0x17c09d9d0>, <great_tables._gt_data.FormatInfo object at 0x17c09dac0>, <great_tables._gt_data.FormatInfo object at 0x17c09dbb0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', 